This notebook uses the `tynt` package, available for download with:

    pip install git+https://github.com/bmorris3/tynt

In [ ]:
import json

import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

import astropy.units as u
from astropy.modeling.models import BlackBody
from astropy.table import Table

In [ ]:
n_wavelengths = 10_000
wavelength = np.logspace(-1, 1, n_wavelengths) * u.um
nu = wavelength.to(u.GHz, u.spectral())[::-1]

In [ ]:
# built-in filter names:
filter_names = [
    "SLOAN/SDSS.u", "Generic/Stromgren.v", "Generic/Johnson.B", 
    "Generic/Bessell.V", "CHEOPS/CHEOPS.band", "GAIA/GAIA2.G", 
    "Kepler/Kepler.K", "TESS/TESS.Red"
]

In [ ]:
from tynt import FilterGenerator, Filter

f = FilterGenerator()

In [ ]:
filters = dict()
for filt_name in filter_names:
    filters[filt_name] = f.download_true_transmittance(filt_name)

In [ ]:
# SOHO VIRGO filter profile described here: 
# https://adsabs.harvard.edu/full/1995ASPC...76..408A
wl_soho = wavelength.copy()
# tr_soho = np.where(np.abs(wl_soho - 500*u.nm) < 2.5 * u.nm, 1, 0)
tr_soho = np.ones_like(wl_soho)

filters['SOHO VIRGO'] = Filter(wl_soho, tr_soho)

In [ ]:
# JWST/NIRSpec filter profile:
wl_nirspec, tr_nirspec = np.loadtxt('../data/NIRSpec_S1600_prism_clear_throughput.csv', unpack=True, delimiter=',')

filters['NIRSpec/Prism'] = Filter(wl_nirspec*u.um, tr_nirspec)

In [ ]:
def compare_filters(filter_names, T_S=5777 * u.K, nu=nu, wavelength=wavelength):
    # assert filter_names[0] == 'SOHO VIRGO'
    # Morris 2020, eqn 11:
    dT = np.atleast_2d([-10, 10]).T * u.K
    temperatures = dT + T_S

    I_nu = BlackBody(T_S)(wavelength)
    dI_dT = np.diff(BlackBody(temperatures)(wavelength), axis=0)[0] / dT.ptp()
    
    f0, f1 = filter_names
    filt0_transmittance = np.interp(
        wavelength, filters[f0].wavelength, filters[f0].transmittance, left=0, right=0
    )
    filt0_transmittance = filt0_transmittance / np.trapz(filt0_transmittance, wavelength)
    filt1_transmittance = np.interp(
        wavelength, filters[f1].wavelength, filters[f1].transmittance, left=0, right=0
    )
    filt1_transmittance = filt1_transmittance / np.trapz(filt1_transmittance, wavelength)

    ratio_0 = (
        np.trapz(dI_dT * wavelength * filt1_transmittance, wavelength) /
        np.trapz(dI_dT * wavelength * filt0_transmittance, wavelength)
    )
    ratio_1 = (
        np.trapz(I_nu * wavelength * filt0_transmittance, wavelength) / 
        np.trapz(I_nu * wavelength * filt1_transmittance, wavelength)
    )
    return ratio_0 * ratio_1

for other in filter_names + ['NIRSpec/Prism']:
    alpha = compare_filters(['SOHO VIRGO', other]).value
    print(f"{other.split('/')[0]} amplitude is {alpha:.2f}x SOHO")

In [ ]:
# Simple sanity check:
assert compare_filters(['SOHO VIRGO', 'SOHO VIRGO']) == 1

In [ ]:
temperatures = np.arange(2000, 6500, 50)
plot_filters = filter_names + ['NIRSpec/Prism']

alpha = np.zeros((len(temperatures), len(plot_filters)))
for i, T_S in enumerate(temperatures):
    for j, other in enumerate(plot_filters):
        alpha[i, j] = compare_filters(['SOHO VIRGO', other], T_S * u.K).value

Fit with a simple parameterization:

In [ ]:
plt.plot(temperatures, alpha, alpha=0.5)
bounds = [[None, None], [1e-3, None], [None, None]]

labels = [pf.split('/')[0] 
          if not (
              pf.split('/')[0].startswith("Generic") or
              pf.split('/')[0].startswith("SLOAN")) 
          else pf.split('/')[1].replace('.', ' ')
          for pf in plot_filters]

rows = []
for i, filt in enumerate(labels):
    #model = lambda p: p[0] * (temperatures - 2000) ** p[1] + p[2]
    model = lambda p: p[0] * np.exp(p[1] * (2000 - temperatures) / 1000) + p[2]
    chi2 = lambda p: np.sum((model(p) - alpha[:, i])**2)
    result = minimize(chi2, [1, 0.1, -1.5], method='l-bfgs-b', bounds=bounds) 
    rows.append([filt] + result.x.tolist())
    plt.plot(temperatures, model(result.x), color=f'C{i}', ls='--')

results_table = Table(rows=rows, names="Filter $c_0$ $c_1$ $c_2$".split())

formats = len(result.x) * ['%.2f']
for col, fmt in zip(results_table.colnames[1:], formats):
    results_table[col].format = fmt
results_table.write('tables/estimate_alpha.tex', format='latex', overwrite=True)

Write out the results to a JSON file: 

In [ ]:
results_pandas = results_table.to_pandas(index='Filter').transpose()

json.dump(
    json.loads(results_pandas.to_json()), 
    open('tables/estimate_alpha.json', 'w'), 
    indent=4
)

In [ ]:
lam_mean = np.ones((len(plot_filters)))

for i, filt in enumerate(plot_filters):
    lam_mean[i] = (
        np.trapz(filters[filt].wavelength * filters[filt].transmittance, filters[filt].wavelength) / 
        np.trapz(filters[filt].transmittance, filters[filt].wavelength)
    ).to(u.nm).value

In [ ]:
labels_with_wavelengths = [
    f"{lam_mean[i]:.0f} nm ({lab})"
    for i, lab in enumerate(labels)
]

In [ ]:
def cmap(lam):
    log_min = np.log(400)
    log_max = np.log(900)
    x = (np.log(lam) - log_min) / (log_max - log_min)
    return plt.cm.Spectral_r(x)

fig, ax = plt.subplots(figsize=(4, 3))
for alpha_i, label, lam in zip(alpha.T, labels_with_wavelengths, lam_mean):
    ax.plot(temperatures, alpha_i, label=label, color=cmap(lam))

ax.legend(title="$\\bar{{\\lambda}}=$", loc=(1.01, 0.15), alignment='left')
for sp in ['right', 'top']:
    ax.spines[sp].set_visible(False)

ax.set(
    xlabel=r'$T_{\rm eff}$ [K]',
    ylabel=r'$\alpha$'
)
fontsize = 14
ax.set_xlabel(ax.get_xlabel(), fontsize=fontsize)
ax.set_ylabel(ax.get_ylabel(), fontsize=fontsize)

fig.savefig('plots/alpha.pdf', bbox_inches='tight')